<a href="https://colab.research.google.com/github/Hanbin-git/Dacon_cacer/blob/main/start(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile

zip_path = '/content/drive/MyDrive/data.zip'
extract_path = '/content/project_data'  # 원하는 경로

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [3]:
import pandas as pd

path = '/content/project_data/'  # 압축 해제 경로
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

print(train.shape, test.shape, submission.shape)


(1681, 3) (100, 2) (100, 2)


In [4]:
# RDKit 설치 (Colab에서 가능)
!pip uninstall -y rdkit-pypi
!pip install rdkit-pypi==2022.9.5 optuna xgboost



Found existing installation: rdkit-pypi 2022.9.5
Uninstalling rdkit-pypi-2022.9.5:
  Successfully uninstalled rdkit-pypi-2022.9.5
  Using cached rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
Using cached rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.4 MB)


In [5]:
# ✅ RDKit + XGBoost + Optuna 전체 파이프라인
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, MACCSkeys, AllChem
import xgboost as xgb
import optuna
import warnings
warnings.filterwarnings("ignore")

# ✅ 경로 설정
path = '/content/project_data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

# ✅ 분자 특성 추출 함수
def get_molecule_descriptors(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None: return [0] * 2232
        basic = [
            Descriptors.MolWt(mol), Descriptors.MolLogP(mol),
            Descriptors.NumHAcceptors(mol), Descriptors.NumHDonors(mol),
            Descriptors.TPSA(mol), Descriptors.NumRotatableBonds(mol),
            Descriptors.NumAromaticRings(mol), Descriptors.NumHeteroatoms(mol),
            Descriptors.FractionCSP3(mol), Descriptors.NumAliphaticRings(mol),
            Lipinski.NumAromaticHeterocycles(mol), Lipinski.NumSaturatedHeterocycles(mol),
            Lipinski.NumAliphaticHeterocycles(mol), Descriptors.HeavyAtomCount(mol),
            Descriptors.RingCount(mol), Descriptors.NOCount(mol),
            Descriptors.NHOHCount(mol), Descriptors.NumRadicalElectrons(mol)
        ]
        morgan = [int(b) for b in AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048).ToBitString()]
        maccs = [int(b) for b in MACCSkeys.GenMACCSKeys(mol).ToBitString()]
        return basic + morgan + maccs
    except:
        return [0] * 2232

# ✅ 피처 생성
train['features'] = train['Canonical_Smiles'].apply(get_molecule_descriptors)
test['features'] = test['Canonical_Smiles'].apply(get_molecule_descriptors)
X_train = np.array(train['features'].tolist())
y_train = train['Inhibition'].values
X_test = np.array(test['features'].tolist())

# ✅ 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ✅ 평가 함수
def normalized_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred)) / (np.max(y_true) - np.min(y_true))
def pearson_correlation(y_true, y_pred):
    return np.clip(np.corrcoef(y_true, y_pred)[0, 1], 0, 1)
def competition_score(y_true, y_pred):
    return 0.5 * (1 - min(normalized_rmse(y_true, y_pred), 1)) + 0.5 * pearson_correlation(y_true, y_pred)

# ✅ Optuna 튜닝
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 1000),
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 2),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 2),
        "random_state": 42
    }
    X_tr, X_val, y_tr, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)
    model = xgb.XGBRegressor(**params)
    model.fit(X_tr, y_tr, verbose=False)
    y_val_pred = model.predict(X_val)
    return -competition_score(y_val, y_val_pred)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

# ✅ 최종 모델 학습 및 예측
final_model = xgb.XGBRegressor(**study.best_params)
final_model.fit(X_train_scaled, y_train)
submission["Inhibition"] = final_model.predict(X_test_scaled)
submission.to_csv("submission_optuna_xgb.csv", index=False)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

...


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found

[I 2025-06-25 08:30:02,507] A new study created in memory with name: no-name-1448a844-f351-4828-88d3-cbf38b798e96
[I 2025-06-25 08:30:15,187] Trial 0 finished with value: -0.5961413612755311 and parameters: {'n_estimators': 831, 'max_depth': 6, 'learning_rate': 0.18696125612297476, 'subsample': 0.7756772162203653, 'colsample_bytree': 0.731277901721106, 'reg_alpha': 1.6019868146241438, 'reg_lambda': 1.5870319831947708}. Best is trial 0 with value: -0.5961413612755311.
[I 2025-06-25 08:30:27,400] Trial 1 finished with value: -0.6167415413789265 and parameters: {'n_estimators': 999, 'max_depth': 9, 'learning_rate': 0.1982812020665366, 'subsample': 0.8952406906007586, 'colsample_bytree': 0.9798881001741764, 'reg_alpha': 1.7532193343259772, 'reg_lambda': 1.8974095921074052}. Best is trial 1 with value: -0.6167415413789265.
[I 2025-06-25 08:30:36,401] Trial 2 finished with value: -0.626186448067624 and parameters: {'n_estimators': 374, 'max_depth': 10, 'learning_rate': 0.04745510580454542, '

In [ ]:


# ✅ 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, MACCSkeys, AllChem
import xgboost as xgb
import optuna
import warnings
warnings.filterwarnings("ignore")

# ✅ 데이터 로딩
path = '/content/project_data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

# ✅ 분자 특성 추출 함수
def get_molecule_descriptors(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None: return [0] * 2232
        basic = [
            Descriptors.MolWt(mol), Descriptors.MolLogP(mol),
            Descriptors.NumHAcceptors(mol), Descriptors.NumHDonors(mol),
            Descriptors.TPSA(mol), Descriptors.NumRotatableBonds(mol),
            Descriptors.NumAromaticRings(mol), Descriptors.NumHeteroatoms(mol),
            Descriptors.FractionCSP3(mol), Descriptors.NumAliphaticRings(mol),
            Lipinski.NumAromaticHeterocycles(mol), Lipinski.NumSaturatedHeterocycles(mol),
            Lipinski.NumAliphaticHeterocycles(mol), Descriptors.HeavyAtomCount(mol),
            Descriptors.RingCount(mol), Descriptors.NOCount(mol),
            Descriptors.NHOHCount(mol), Descriptors.NumRadicalElectrons(mol)
        ]
        morgan = [int(b) for b in AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048).ToBitString()]
        maccs = [int(b) for b in MACCSkeys.GenMACCSKeys(mol).ToBitString()]
        return basic + morgan + maccs
    except:
        return [0] * 2232

# ✅ 피처 생성
train['features'] = train['Canonical_Smiles'].apply(get_molecule_descriptors)
test['features'] = test['Canonical_Smiles'].apply(get_molecule_descriptors)
X = np.array(train['features'].tolist())
y = train['Inhibition'].values
X_test = np.array(test['features'].tolist())

# ✅ 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# ✅ 평가 함수
def normalized_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred)) / (np.max(y_true) - np.min(y_true))
def pearson_correlation(y_true, y_pred):
    return np.clip(np.corrcoef(y_true, y_pred)[0, 1], 0, 1)
def competition_score(y_true, y_pred):
    return 0.5 * (1 - min(normalized_rmse(y_true, y_pred), 1)) + 0.5 * pearson_correlation(y_true, y_pred)

# ✅ Optuna 튜닝 (1회만, 전체 데이터 기반)
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 1000),
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 2),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 2),
        "random_state": 42
    }
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    model = xgb.XGBRegressor(**params)
    model.fit(X_tr, y_tr, verbose=False)
    y_val_pred = model.predict(X_val)
    return -competition_score(y_val, y_val_pred)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)
best_params = study.best_params

# ✅ KFold 앙상블 학습 + 예측
kf = KFold(n_splits=5, shuffle=True, random_state=42)
test_preds = np.zeros(X_test.shape[0])
scores = []

for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
    print(f"📂 Fold {fold + 1}")
    X_tr, X_val = X[tr_idx], X[val_idx]
    y_tr, y_val = y[tr_idx], y[val_idx]

    model = xgb.XGBRegressor(**best_params)
    model.fit(X_tr, y_tr, verbose=False)
    y_val_pred = model.predict(X_val)
    score = competition_score(y_val, y_val_pred)
    print(f"Fold {fold + 1} Score: {score:.4f}")
    scores.append(score)

    test_preds += model.predict(X_test) / 5  # 앙상블 평균

# ✅ 후처리: 0~100 클리핑 + float32로 저장
test_preds = np.clip(test_preds, 0, 100)
submission['Inhibition'] = test_preds.astype(np.float32)
submission.to_csv('submission_kfold_optuna.csv', index=False)

print(f"\n✅ 평균 Score: {np.mean(scores):.4f}")
print("📁 제출파일 저장 완료: submission_kfold_optuna.csv")


[I 2025-06-25 08:34:31,767] A new study created in memory with name: no-name-dc8bc1bf-6a66-4ec2-8b4c-b385804c9bd6
[I 2025-06-25 08:34:43,992] Trial 0 finished with value: -0.6050533160267576 and parameters: {'n_estimators': 614, 'max_depth': 7, 'learning_rate': 0.06384288842400268, 'subsample': 0.7496083557773568, 'colsample_bytree': 0.8101318327657878, 'reg_alpha': 0.3062655545012618, 'reg_lambda': 0.7473706297748046}. Best is trial 0 with value: -0.6050533160267576.
[I 2025-06-25 08:34:56,077] Trial 1 finished with value: -0.6285336178586667 and parameters: {'n_estimators': 906, 'max_depth': 6, 'learning_rate': 0.12619509070647225, 'subsample': 0.8856617599915513, 'colsample_bytree': 0.8492895406038632, 'reg_alpha': 1.7317536329101884, 'reg_lambda': 0.8717455718289042}. Best is trial 1 with value: -0.6285336178586667.
[I 2025-06-25 08:35:00,503] Trial 2 finished with value: -0.6262705667340356 and parameters: {'n_estimators': 351, 'max_depth': 8, 'learning_rate': 0.04077408775798576,

📂 Fold 1
Fold 1 Score: 0.6384
📂 Fold 2


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, MACCSkeys, AllChem
import warnings
warnings.filterwarnings("ignore")

# ✅ 경로
path = "/content/drive/MyDrive/data/"
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
submission = pd.read_csv(path + "sample_submission.csv")

# ✅ Feature 추출 함수
def get_molecule_descriptors(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return [0] * 2232

        basic_descriptors = [
            Descriptors.MolWt(mol),
            Descriptors.MolLogP(mol),
            Descriptors.NumHAcceptors(mol),
            Descriptors.NumHDonors(mol),
            Descriptors.TPSA(mol),
            Descriptors.NumRotatableBonds(mol),
            Descriptors.NumAromaticRings(mol),
            Descriptors.NumHeteroatoms(mol),
            Descriptors.FractionCSP3(mol),
            Descriptors.NumAliphaticRings(mol),
            Lipinski.NumAromaticHeterocycles(mol),
            Lipinski.NumSaturatedHeterocycles(mol),
            Lipinski.NumAliphaticHeterocycles(mol),
            Descriptors.HeavyAtomCount(mol),
            Descriptors.RingCount(mol),
            Descriptors.NOCount(mol),
            Descriptors.NHOHCount(mol),
            Descriptors.NumRadicalElectrons(mol),
        ]

        morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        morgan_features = [int(bit) for bit in morgan_fp.ToBitString()]
        maccs_fp = MACCSkeys.GenMACCSKeys(mol)
        maccs_features = [int(bit) for bit in maccs_fp.ToBitString()]

        return basic_descriptors + morgan_features + maccs_features
    except:
        return [0] * 2232

# ✅ Feature 생성
train['features'] = train['Canonical_Smiles'].apply(get_molecule_descriptors)
test['features'] = test['Canonical_Smiles'].apply(get_molecule_descriptors)
X = np.array(train['features'].tolist())
y = train['Inhibition'].values
X_test = np.array(test['features'].tolist())

# ✅ 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# ✅ Metric
def normalized_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred)) / (np.max(y_true) - np.min(y_true))

def pearson_correlation(y_true, y_pred):
    return np.clip(np.corrcoef(y_true, y_pred)[0,1], 0, 1)

def competition_score(y_true, y_pred):
    return 0.5 * (1 - normalized_rmse(y_true, y_pred)) + 0.5 * pearson_correlation(y_true, y_pred)

# ✅ 모델 정의
model = LGBMRegressor(
    n_estimators=500, learning_rate=0.05,
    max_depth=6, num_leaves=31,
    subsample=0.8, colsample_bytree=0.8,
    random_state=42
)

# ✅ 5-Fold 교차검증 + 예측
kf = KFold(n_splits=5, shuffle=True, random_state=42)
val_scores = []
test_preds = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    X_train_fold, X_val_fold = X_scaled[train_idx], X_scaled[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    model.fit(X_train_fold, y_train_fold)
    val_pred = model.predict(X_val_fold)
    score = competition_score(y_val_fold, val_pred)
    val_scores.append(score)
    print(f"[Fold {fold+1}] 검증 점수: {score:.4f}")

    test_pred = model.predict(X_test_scaled)
    test_preds.append(test_pred)

# ✅ 앙상블 (평균)
final_test_pred = np.mean(test_preds, axis=0)

# ✅ 후처리: 0~100 클리핑
final_test_pred = np.clip(final_test_pred, 0, 100)

# ✅ 제출 파일 저장
submission['Inhibition'] = final_test_pred
submission.to_csv("submission_kfold_lgbm.csv", index=False)
print("✅ 제출 파일 저장 완료: submission_kfold_lgbm.csv")
print(f"평균 검증 점수: {np.mean(val_scores):.4f}")
